In [11]:

from geopy.geocoders import Nominatim

import requests
import numpy as np
from joblib import load
import xgboost as xgb
import pandas as pd


def get_weather_forecast(url):
    res = requests.get(url)
    return res.json()

def get_city_location(city):

    geolocator = Nominatim(user_agent="MyApp")
    location = geolocator.geocode(city)
    return location.latitude, location.longitude

def preprocess_response(res):

    # Pressure
    press = (res["hourly"]["pressure_msl"])
    daily_slp = [] 
    for i in range(7):
        daily_slp.append(np.array(press[24*(i):24*(i+1)]).mean())

    #  Temprature
    temp = res["hourly"]["temperature_2m"]
    daily_temp = [] 
    for i in range(7):
        daily_temp.append(np.array(temp[24*(i):24*(i+1)]).mean())

    # Dewpoint
    dewp = res["hourly"]["dewpoint_2m"]
    daily_dewp = [] 
    for i in range(7):
        daily_dewp.append(np.array(dewp[24*(i):24*(i+1)]).mean())

    # Windspeed
    wspd = res["hourly"]["windspeed_10m"]
    daily_wspd = [] 
    for i in range(7):
        daily_wspd.append(np.array(wspd[24*(i):24*(i+1)]).mean())

    # Cloudcover
    cldcover = res["hourly"]["cloudcover_low"]
    daily_visibility = [] 
    vals = []
    for i in range(7):
        val = 100 - (np.array(cldcover[24*(i):24*(i+1)]).mean())
        vals.append(val)
        new_value = ( (val - 0) / (100 - 0) ) * (6.20 - 0.6) + 0.6
        daily_visibility.append(new_value)

    print("===========================")
    print(vals)
    print(daily_visibility)

    # Weathercode
    wcode = res["hourly"]["weathercode"]

    # Fog
    daily_fog_indicator = []
    fog_codes = [45, 48]
    for i in range(7):
        daily_fog_indicator.append(any(item in fog_codes for item in np.array(wcode[24*(i):24*(i+1)])))

    # Drizzle
    drizzle_codes = [51, 53, 55, 56, 57]
    daily_drizzle_indicator = []
    for i in range(7):
        daily_drizzle_indicator.append(any(item in drizzle_codes for item in np.array(wcode[24*(i):24*(i+1)])))

    # Snow
    snow_codes = [71, 73, 75, 77]
    daily_snow_indicator = []
    for i in range(7):
        daily_snow_indicator.append(any(item in snow_codes for item in np.array(wcode[24*(i):24*(i+1)])))

    # Thunder
    thunder_codes = [95, 96, 99]
    daily_thunder_indicator = []
    for i in range(7):
        daily_thunder_indicator.append(any(item in thunder_codes for item in np.array(wcode[24*(i):24*(i+1)])))

    daily_max_wspd = res["daily"]["windspeed_10m_max"]

    daily_max_wgust = res["daily"]["windgusts_10m_max"]

    daily_max_temp = res["daily"]["temperature_2m_max"]

    daily_min_temp = res["daily"]["temperature_2m_min"]

    daily_prcp = res["daily"]["precipitation_sum"]

    # All days weather
    days_weather = []
    for i in range (7):
        days_weather.append(np.array([daily_temp[i], daily_dewp[i], daily_slp[i], 996.125888, daily_visibility[i], daily_wspd[i], daily_max_wspd[i], daily_max_wgust[i], daily_max_temp[i], daily_min_temp[i], daily_prcp[i], int(daily_fog_indicator[i]), int(daily_drizzle_indicator[i]), int(daily_snow_indicator[i]), int(daily_thunder_indicator[i])]))

    original_days_weather = days_weather
    sc=load('/home/lotfy/Documents/Work/DFKI/PAIRS_Demonstrator/backend/forecast/scaler.bin')
    days_weather = sc.transform(days_weather)
    return days_weather, original_days_weather


In [13]:
city = "Raubling"
# Get city location
lat, long = get_city_location(city)

#  Weather API 
url = 'https://api.open-meteo.com/v1/forecast?latitude={}&longitude={}&hourly=cloudcover_low,weathercode,windspeed_10m,temperature_2m,pressure_msl,dewpoint_2m&daily=temperature_2m_max,temperature_2m_min,precipitation_sum,rain_sum,showers_sum,snowfall_sum,precipitation_hours,weathercode,windspeed_10m_max,windgusts_10m_max&temperature_unit=fahrenheit&current_weather=true&windspeed_unit=kn&precipitation_unit=inch&timezone=CET'.format(lat, long)
res = get_weather_forecast(url)
data, data_original = preprocess_response(res)

features_names = ['temp', 'dwep', 'slp', 'stp', 'visibility', 'wspd', 'mxwspd', 'mxwgust', 'mxtemp', 'mintemp', 'prcp', 'fog', 'drizzle', 'snow', 'thunder']
# Load model
model = xgb.XGBClassifier()
model.load_model("/home/lotfy/Documents/Work/DFKI/PAIRS_Demonstrator/backend/forecast/model.json")

# Predict
preds = model.predict(data)
prob = model.predict_proba(data)

confidence = []
for i in range(len(prob)):
    if preds[i] == 1 and prob[i][1] < 0.85:
        preds[i] = 0
        prob[i][0] = ((prob[i][0] - 0) / (1 - 0)) * (1 - 0.5) + 0.5
        confidence.append(prob[i][0])
        continue
    elif preds[i] == 1 and prob[i][1] >= 0.85:
        new_value = ((prob[i][1] - 0.50) / (1 - 0.5) ) * (0.9 - 0.3) + 0.3
        confidence.append(new_value)
        continue
    else:
        new_value = ((prob[i][preds[i]] - 0.50) / (1 - 0.5) ) * (0.96 - 0.5) + 0.5
        confidence.append(new_value)
print(preds)
print("--------------")
print(confidence)
print("--------------")
print(data_original)

[100.0, 97.41666666666667, 90.45833333333333, 73.375, 86.375, 96.625, 65.54166666666666]
[6.2, 6.0553333333333335, 5.665666666666667, 4.7090000000000005, 5.437, 6.011, 4.270333333333332]
[0 0 0 0 0 1 0]
--------------
[0.958297827243805, 0.959473352432251, 0.9468147277832031, 0.9286780142784119, 0.7308443689346313, 0.833288598060608, 0.9593980622291565]
--------------
[array([  63.        ,   48.8       , 1018.30416667,  996.125888  ,
          6.2       ,    3.59583333,    9.1       ,   18.7       ,
         75.7       ,   48.9       ,    0.        ,    0.        ,
          0.        ,    0.        ,    0.        ]), array([6.37000000e+01, 4.93833333e+01, 1.01638333e+03, 9.96125888e+02,
       6.05533333e+00, 2.33333333e+00, 7.00000000e+00, 1.46000000e+01,
       7.97000000e+01, 4.60000000e+01, 0.00000000e+00, 1.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00]), array([6.70333333e+01, 5.63958333e+01, 1.01138333e+03, 9.96125888e+02,
       5.66566667e+00, 2.0125000

In [14]:
predict_fn = lambda x: rf.predict_proba(data[0])

In [15]:
import sklearn
import sklearn.datasets
import sklearn.ensemble
import numpy as np
import lime
import lime.lime_tabular
from __future__ import print_function
np.random.seed(1)

In [6]:
!pip install lime

     |████████████████████████████████| 275 kB 249 kB/s eta 0:00:01
  Created wheel for lime: filename=lime-0.2.0.1-py3-none-any.whl size=283843 sha256=ab71383055bc86b9b79132b3865dbaf78250a113aac512b945ce4c16bba15a1c
  Stored in directory: /home/lotfy/.cache/pip/wheels/ed/d7/c9/5a0130d06d6310bc6cbe55220e6e72dcb8c4eff9a478717066
Successfully built lime


In [16]:
explainer = lime.lime_tabular.LimeTabularExplainer(None, feature_names=features_names, class_names=["Outage", "No Outage"])

AttributeError: 'NoneType' object has no attribute 'shape'